<span style="color:blue">Copyright (c) 2014-2020 National Technology and Engineering
Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525
with National Technology and Engineering Solutions of Sandia, LLC,
the U.S. Government retains certain rights in this software.</span>    
    
<span style="color:blue">Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.</span>    
    
<span style="color:blue">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

In [1]:
import csv
import datetime
import itertools
import math
import os.path
import random
import sys

from tracktable.core import geomath
from tracktable.domain import terrestrial
from tracktable.info import cities
from tracktable.source.point import random_circle_linear_falloff


n_largest_cities(howmany: int) -> list of CityInfo objects    
    
Retrieve a list of the N largest cities in the world (by population) sorted in descending order.    

In [2]:
def n_largest_cities(howmany):
    return cities.largest_cities_in_bbox(count=howmany)

Point radius for city(city: tracktable.info.cities,CityInfo) -> float(km)    
    
Return a radius proportional to a city's population. Arbitrarily, a city with a population of 1 million will get a radius of 50km.    
    
This has no particular real-world meaning. It's just a way to scatter points around the city center.

In [3]:
def point_radius_for_city(city):
    return (50 * city.population/1000000)

In [4]:
def write_points_to_file(point_source, outfile):
    writer = terrestrial.TrajectoryPointWriter(outfile)
    writer.write(point_source)
    

In [5]:
def points_near_city(city, num_points):
    center = terrestrial.TrajectoryPoint(city.longitude, city.latitude)
    center.object_id = 'ANON'
    center.timestamp = datetime.datetime.now()
    radius = point_radius_for_city(city)

    return random_circle_linear_falloff(center, 
                                        num_points=num_points, 
                                        max_radius_km=radius)
    

In [6]:
num_cities = 100
num_points_per_city = 1000
outfilename = os.path.expanduser('~/city_points.csv')

print("INFO: Generating {} points around each of the {} largest cities in the world.".format(num_points_per_city, num_cities))
all_sources = [ points_near_city(city, num_points_per_city) for city in n_largest_cities(num_cities) ]
combined_point_source = itertools.chain(*all_sources)
    
with open(outfilename, 'wb') as outfile:
    write_points_to_file(combined_point_source, outfile)


INFO: Generating 1000 points around each of the 100 largest cities in the world.
DEBUG: cities_in_bbox: bbox_min is (-180, -90), bbox_max is (180, 90)
DEBUG: min_longitude is -180, min_latitude is -90, max_longitude is 180, max_latitude is 90
